<a href="https://colab.research.google.com/github/AnasHamoutni/UpScale-Resolution-Colorize-Videos/blob/main/UpScale_Resolution_%26_Colorize_Videos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ◢ UpScale Resolution & Colorize your own videos!


My name is **ANAS HAMOUTNI**, a Data Scientist from Morocco, and this is my second Data Science Project on Github and is intended as a tool to UpScale Resolution and Colorize gifs and short videos, if you are trying to convert longer video on Colab you may hit the limit on processing space. Running the Jupyter notebook on your own machine is recommended (and faster) for larger video sizes.

**Credits** are given to Jantic (Deoldify project) and all the people who worked on this Subject.

In [18]:
#@title ##**Install all the necessary libraries** { display-mode: "form" }

!pip install youtube_dl
!pip install ffmpeg
!pip install ffmpeg-python
!pip install torchvision==0.5
!pip install torch==1.4

from IPython.display import clear_output
from google.colab import files
import imageio
import youtube_dl
import cv2
import os
import torch
import fastai
import ffmpeg
import os.path as osp
import logging
import shutil
import re
import gc
from PIL import Image
from tqdm import tqdm
from os import path
import numpy as np
import moviepy.editor as mpy
from moviepy.video.io.ffmpeg_writer import FFMPEG_VideoWriter
from pathlib import Path
import sys
import glob
from IPython import display as ipythondisplay
from IPython.display import Image as ipythonimage
torch.backends.cudnn.benchmark=True
%matplotlib inline

!rm -rf sample_data
#@title ##**Clone the repository and download the necessary components** { display-mode: "form" }
%cd /content
!git clone https://github.com/xinntao/ESRGAN.git
!cp -r video.mp4 /content/ESRGAN/
%cd /content/ESRGAN
!git checkout tags/old-arch
model_url = "https://www.dropbox.com/s/vouc15j8jjp2o5n/RRDB_ESRGAN_x4_old_arch.pth?dl=0"
!wget $model_url --content-disposition -P models
import architecture as arch
import os.path
!mkdir frames
!rm -rf results/baboon_ESRGAN.png
#@title ##**Clone the repository and install the necessary dependencies** { display-mode: "form" }
%cd /content
!git clone https://github.com/jantic/DeOldify.git
%cd /content/DeOldify
!pip install -r colab_requirements.txt
clear_output()

In [ ]:
#@title ##**Upload video** { display-mode: "form" }
#@markdown *Enter a link to the video below (for example, YouTube or Twitter), or leave the **source_url** field blank (in this case, you will be asked to upload the video from your computer).*
source_url = 'https://www.youtube.com/watch?v=ZB-HI9fQBE0' #@param {type:"string"}

if source_url == '':
  uploaded = files.upload()
  for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(
        name=fn, length=len(uploaded[fn])))
  os.rename(fn, fn.replace(" ", ""))
  fn = fn.replace(" ", "")
  file_name = "downloaded_video." + fn.split(".")[-1]
  !mv -f $fn $file_name

else:
  try:
    ydl_opts = {
        'format': 'bestvideo[ext=mp4]+bestaudio[ext=m4a]/mp4',
        'outtmpl': 'downloaded_video.mp4',
        }
    with youtube_dl.YoutubeDL(ydl_opts) as ydl:
      ydl.download([source_url])
    file_name = 'downloaded_video.mp4'
  
  except BaseException:
    !wget $source_url
    fn = source_url.split('/')[-1]
    os.rename(fn, fn.replace(" ", ""))
    fn = fn.replace(" ", "")
    file_name = "downloaded_video." + fn.split(".")[-1]
    !mv -f $fn $file_name

!cp -r downloaded_video.mp4 video.mp4
clear_output()
fps_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FPS))
frames_of_video = int(cv2.VideoCapture(file_name).get(cv2.CAP_PROP_FRAME_COUNT))
#@markdown *Downloading videos lasting longer than one minute is not recommended. In addition, don't upload video that have a "space" or a "dot" in the title.*

#@markdown *If there is an error during execution, then run this block again*

In [ ]:
#@title ##**Split the video into frames** { display-mode: "form" }
%cd /content/ESRGAN/
!mkdir frames

frames_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FRAME_COUNT))
fps_of_video = int(cv2.VideoCapture("video.mp4").get(cv2.CAP_PROP_FPS))
vidcap = cv2.VideoCapture('video.mp4')
success,image = vidcap.read()
count = 0
success = True
while success:
  cv2.imwrite("frames/frame%09d.jpg" % count, image)
  success,image = vidcap.read()
  count += 1


/content/ESRGAN


In [19]:
#@title ##**Upscale resolution** { display-mode: "form" }
#@markdown **How many times to upscale the resolution:**
upscale = 4 #@param {type: "slider", min: 4, max: 8}
%cd /content/ESRGAN
%env CUDA_VISIBLE_DEVICES=0
device = torch.device('cuda')
model = arch.RRDB_Net(3, 3, 64, 23, gc=32, upscale=upscale, norm_type=None, act_type='leakyrelu', \
                        mode='CNA', res_scale=1, upsample_mode='upconv')
model.load_state_dict(torch.load('models/{:s}'.format('RRDB_ESRGAN_x4_old_arch.pth')), strict=True)
model.eval()
for k, v in model.named_parameters():
    v.requires_grad = False
model = model.to(device)

count_frames = 0

for path in glob.glob('frames/*'):
    base = os.path.splitext(os.path.basename(path))[0]
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    path = 'results/{:s}_rlt.png'.format(base)
    cv2.imwrite(path, output)
    count_frames += 1
    clear_output()
    print("Processed: {} из {}".format(str(count_frames), str(frames_of_video)))
clear_output()

In [ ]:
#@title ##**Collecting a video file** { display-mode: "form" }
import os
import shutil
import imageio
shutil.rmtree('/content/ESRGAN/frames', ignore_errors=True)
%cd /content/ESRGAN/

frames = []
img = os.listdir('results/')
img.sort()
for i in img:
  frames.append(imageio.imread("results/"+i))
frames = np.array(frames)
imageio.mimsave("upscaled_video.mp4", frames, fps=30)

!cp -r upscaled_video.mp4 /content/upscaled_video.mp4
clear_output()

In [ ]:
#@title ##**Get result** { display-mode: "form" }
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/upscaled_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  !zip -r /content/results.zip /content/ESRGAN/results
  from google.colab import files
  files.download("/content/results.zip")


In [ ]:
#@title ##**Upload the pre-trained model** { display-mode: "form" }
from deoldify.visualize import *
!mkdir 'models'
try:
  !rm -rf models/ColorizeVideo_gen.pth
  !gdown https://drive.google.com/uc?id=1-3ONnPYcX9fOqnY-pUKYGJOd6XeFon9X -O ./models/ColorizeVideo_gen.pth
except BaseException:
  !rm -rf models/ColorizeVideo_gen.pth
  !wget https://www.dropbox.com/s/336vn9y4qwyg9yz/ColorizeVideo_gen.pth?dl=0 -O ./models/ColorizeVideo_gen.pth
colorizer = get_video_colorizer()

In [20]:
#@title ##**Colorize the video** { display-mode: "form" }
#@markdown **21, 35** *- optimal values*
!wget https://data.deepai.org/deoldify/ColorizeVideo_gen.pth -O ./models/ColorizeVideo_gen.pth
!rm -rf video
!mkdir 'video'
!mkdir 'video/source'
!cp -r /content/video.mp4 video/source/video.mp4
colorizer = get_video_colorizer()
render_factor = 21  #@param {type: "slider", min: 5, max: 44}
video_path = colorizer.colorize_from_file_name('video.mp4', render_factor)
clear_output()

In [ ]:
#@title ##**Get result** { display-mode: "form" }
!cp -r video/result/video.mp4 /content/colorized_video.mp4
what_next = 'play' #@param ["play", "download"]
if what_next == "play":
  display(mpy.ipython_display("/content/colorized_video.mp4", height=400, autoplay=1, loop=1, maxduration=600))
else:
  files.download('/content/colorized_video.mp4')

In [ ]:
#@title ###**Optional block!** { display-mode: "form" }
#@markdown *If it seems to you that the quality of coloring is insufficient, then run this block, wait until it is complete and look at what value of **render_factor** you like the picture more. Next, run the previous coloring block again, setting the desired value.*
for i in range(10,45,2):
    colorizer.vis.plot_transformed_image('video/bwframes/video/00001.jpg', render_factor=i, display_render_factor=True, figsize=(8,8))